# Tetris player

an RL model that can play Tetris game from (https://play.tetris.com/)

In [2]:
# !uv add pandas numpy matplotlib seaborn scikit-learn jupyter
# !uv add torch torchvision torchmetrics
# !uv add flask flask-cors

# Websocket based exec

## Client-Side: data send

run it in console
```js
(function () {
    const WS_SERVER = 'ws://localhost:8766';
    let ws = null;
    let reconnectTimeout = null;
    const PING_INTERVAL = 5_000; // 10 seconds
    let pingInterval = null;

    function connect() {
        ws = new WebSocket(WS_SERVER);

        ws.onopen = () => {
            console.log('[RemoteExec] Connected to WebSocket server');
            startPing();
        };

        ws.onmessage = async (event) => {
            try {
                const data = JSON.parse(event.data);

                if (data.type === 'pong') {
                    // Keep-alive response, nothing to do
                    return;
                }

                if (data.type === 'execute') {
                    const code = data.code;

                    // Wrap user code in an async function
                    let result;
                    try {
                        // Use new Function to avoid eval restrictions and enable async
                        const fn = new Function('return (async () => {' + code + '})()');
                        result = fn(); // This returns a Promise
                    } catch (e) {
                        // Syntax error or function creation failed
                        sendResult({
                            type: 'execution_result',
                            uid: data.uid,
                            result: null,
                            error: '[JS Parse Error] ' + (e.stack || e.toString()),
                        });
                        return;
                    }

                    // Handle Promise resolution
                    Promise.resolve(result)
                        .then(value => {
                            // Serialize result safely
                            try {
                                sendResult({
                                    type: 'execution_result',
                                    uid: data.uid,
                                    result: value,
                                    error: null,
                                });
                            } catch (e) {
                                sendResult({
                                    type: 'execution_result',
                                    uid: data.uid,
                                    result: null,
                                    error: '[Serialize Error] ' + (e.stack || e.toString()),
                                });
                            }
                        })
                        .catch(err => {
                            sendResult({
                                type: 'execution_result',
                                uid: data.uid,
                                error: String(err),
                                result: null,
                            });
                        });
                }
            } catch (e) {
                console.error('[RemoteExec] Error processing message:', e);
            }
        };

        ws.onclose = () => {
            console.log('[RemoteExec] Disconnected from WebSocket server');
            stopPing();
            scheduleReconnect();
        };

        ws.onerror = (error) => {
            console.error('[RemoteExec] WebSocket error:', error);
        };
    }

    function sendResult(payload) {
        if (ws && ws.readyState === WebSocket.OPEN) {
            ws.send(JSON.stringify(payload));
        }
    }

    function startPing() {
        stopPing();
        pingInterval = setInterval(() => {
            if (ws && ws.readyState === WebSocket.OPEN) {
                ws.send(JSON.stringify({
                    type: 'ping',
                    uid: crypto.randomUUID(),
                    timestamp: Date.now()
                }));
            }
        }, PING_INTERVAL);
    }

    function stopPing() {
        if (pingInterval) {
            clearInterval(pingInterval);
            pingInterval = null;
        }
    }

    function scheduleReconnect() {
        if (reconnectTimeout) {
            clearTimeout(reconnectTimeout);
        }
        reconnectTimeout = setTimeout(() => {
            console.log('[RemoteExec] Reconnecting...');
            connect();
        }, 3000);
    }

    // Start connection
    connect();
})();
```

## Server

In [1]:
import sys
from uuid import uuid5

import websockets
import json
import time
import threading
from collections import defaultdict
import asyncio
import uuid


class WebSocketServer:
    def __init__(self, host="localhost", port=8766):
        self.host = host
        self.port = port
        self.clients = {}  # {uid: websocket}
        self.client_uid_counter = 0
        self.client_last_seen = {}  # {uid: timestamp}
        self.TIMEOUT = 45  # seconds
        self.msg_callbacks = {}  # {msg_id: callback}
        self.loop = None

    async def handle_client(self, websocket):
        # Assign unique ID
        self.client_uid_counter += 1
        client_uid = self.client_uid_counter
        self.clients[client_uid] = websocket
        self.client_last_seen[client_uid] = time.time()
        print(f"Client {client_uid} connected", flush=True, file=sys.stdout)

        try:
            async for message in websocket:
                try:
                    print(f"From {client_uid}: {message}", flush=True)
                    data = json.loads(message)

                    # validation
                    """
                    {
                      type: "ping" | "pong" | "execute" | "execution_result"
                      uid: str | int
                      timestamp: number (required only for 'ping'/'pong' messages)
                      result: any (required only for 'execution_result' messages | if no error)
                      error: any (required only for 'execution_result' messages | only if error occurs)
                      code: str (required only for 'execute' messages)
                    }
                    """
                    assert isinstance(data, dict), "Data must be a dictionary"
                    assert 'type' in data, "Data must contain 'type' field"
                    assert data['type'] in ['ping', 'pong', 'execute',
                                            'execution_result'], f"Invalid type: {data['type']}"
                    assert 'uid' in data, "Data must contain 'uid' field"
                    assert isinstance(data['uid'], (str, int)), "UID must be string or int"
                    if data['type'] in ['ping', 'pong']:
                        assert 'timestamp' in data, f"Timestamp required for {data['type']}"
                        assert isinstance(data['timestamp'], (int, float)), "Timestamp must be numeric"
                    if data['type'] == 'execute':
                        assert 'code' in data, "Code required for execute"
                        assert isinstance(data['code'], str), "Code must be string"
                    if data['type'] == 'execution_result':
                        assert 'result' in data, "Result required for execution_result"
                        assert 'error' in data, "Error required for execution_result"

                    await self.handle_message(data, client_uid)
                except json.JSONDecodeError:
                    print(f"Invalid JSON from client {client_uid}")
                except AssertionError as e:
                    print(f"Warning: Invalid message format from client {client_uid}: {e}")
        except websockets.exceptions.ConnectionClosed:
            print(f"Client {client_uid} disconnected")
        finally:
            if client_uid in self.clients:
                del self.clients[client_uid]
                if client_uid in self.client_last_seen:
                    del self.client_last_seen[client_uid]

    async def handle_message(self, data, client_uid):
        msg_type = data.get('type')
        msg_id = data.get('uid')

        # Update last seen time
        self.client_last_seen[client_uid] = time.time()

        if msg_type == 'ping':
            await self.send_message(data={'uid': msg_id, 'type': 'pong', 'timestamp': time.time()},
                                    client_uid=client_uid)

        elif msg_type == 'execution_result':
            if msg_id in self.msg_callbacks:
                callback = self.msg_callbacks.pop(msg_id)
                asyncio.create_task(callback(data))

        else:
            print("Warn: msg has no callback!")

    async def send_message(self, data, callback=None, client_uid=None):
        client_uids = [client_uid] if client_uid is not None else self.clients.keys()

        for client_uid in client_uids:
            if client_uid not in self.clients:
                continue

            try:
                # Add message ID if not present
                if 'uid' not in data:
                    data['uid'] = str(uuid.uuid4())

                # Register callback if provided
                if callback is not None:
                    self.msg_callbacks[data['uid']] = callback

                await self.clients[client_uid].send(json.dumps(data))
                print(f"Sent message to client {client_uid}: {data}")  # Log sent message
            except websockets.exceptions.ConnectionClosed:
                # Clean up dead connection
                del self.clients[client_uid]
                if client_uid in self.client_last_seen:
                    del self.client_last_seen[client_uid]

    async def __cleanup_dead_connections(self):
        while True:
            current_time = time.time()
            dead_clients = [
                client_uid for client_uid, last_seen in self.client_last_seen.items()
                if current_time - last_seen > self.TIMEOUT
            ]

            for client_uid in dead_clients:
                print(f"Client {client_uid} timed out")
                if client_uid in self.clients:
                    await self.clients[client_uid].close()
                    del self.clients[client_uid]
                if client_uid in self.client_last_seen:
                    del self.client_last_seen[client_uid]

            await asyncio.sleep(5)  # Check every 5 seconds

    def start_server(self):
        async def run_server():
            server = await websockets.serve(self.handle_client, self.host, self.port)
            print(f"WebSocket server started on ws://{self.host}:{self.port}")

            # Start cleanup task
            cleanup_task = asyncio.create_task(self.__cleanup_dead_connections())

            await server.wait_closed()

        # Create a new event loop for this thread
        self.loop = asyncio.new_event_loop()
        asyncio.set_event_loop(self.loop)
        self.loop.run_until_complete(run_server())

    async def execute_js(self, code, timeout=0.01):
        # Generate a unique message ID for this execution
        msg_id = str(uuid.uuid4())

        # Create a future to wait for the response
        future = asyncio.Future()

        # Define the callback that will set the result of the future
        async def callback(data):
            if not future.done():
                future.set_result(data)

        # Send the execute message with the callback
        await self.send_message(
            data={
                "type": "execute",
                "code": code,
                "uid": msg_id
            },
            callback=callback
        )

        # Wait for the callback to be called and return the result
        result = await asyncio.wait_for(future, timeout=timeout)
        return result

In [2]:
server = WebSocketServer()

server_thread = threading.Thread(target=lambda: server.start_server(), daemon=True)
server_thread.start()

time.sleep(3)  # just wait, to let the client connect

WebSocket server started on ws://localhost:8766
Client 1 connected


In [6]:
await server.execute_js("return 2 + 58")

Sent message to client 1: {'type': 'execute', 'code': 'return 2 + 58', 'uid': '391fd0b6-e18c-4f71-9462-9ffb87b58785'}
From 1: {"type":"execution_result","uid":"391fd0b6-e18c-4f71-9462-9ffb87b58785","result":60,"error":null}


{'type': 'execution_result',
 'uid': '391fd0b6-e18c-4f71-9462-9ffb87b58785',
 'result': 60,
 'error': None}

- [ ] Take canvas to image (png maybe?)
- [ ] Continuesly take canvas img -> capture when images changes
- [ ] then, send the changed image via websocket
    - [ ] implement on client-side
    - [ ] implement on server (just print the uniq images)
- [ ] extract info from the images
    - [ ] level
    - [ ] score
    - [ ] lines
    - [ ] next 3 pieces
    - [ ] held piece
    - [ ] ground pieces (a matrix representing all filled cells without the current piece)
    - [ ] current piece
    - [ ]
- [ ] plug the data into rl model
- [ ] tinker with it
- [ ]

# WebRTC based screen share and game state extractor

In [2]:
!uv add aiortc opencv-python av nest-asyncio

Resolved 167 packages in 1.68s                                       
Prepared 1 package in 38.47s                                             ⠋ Preparing packages... (0/0)                                                   
Installed 1 package in 34ms0.86                             
 + opencv-python==4.11.0.86


### Client
```javascript
(async function startStream() {
    const canvas = document.getElementById('GameCanvas');
    if (!canvas) {
        console.warn("Canvas not found, retrying...");
        return null;
    }

    const stream = canvas.captureStream(15);
    const pc = new RTCPeerConnection();

    stream.getTracks().forEach(track => pc.addTrack(track, stream));

    pc.oniceconnectionstatechange = () => {
        console.log("ICE state:", pc.iceConnectionState);
        if (pc.iceConnectionState === "failed" || pc.iceConnectionState === "disconnected") {
            // Will trigger retry in main loop
        }
    };

    pc.onconnectionstatechange = () => {
        console.log("Connection state:", pc.connectionState);
    };

    try {
        const offer = await pc.createOffer();
        await pc.setLocalDescription(offer);

        const response = await fetch('http://localhost:8081/offer', {
            method: 'POST',
            headers: {'Content-Type': 'application/json'},
            body: JSON.stringify({
                sdp: pc.localDescription.sdp,
                type: pc.localDescription.type
            })
        });

        if (!response.ok) {
            throw new Error(`HTTP ${response.status}`);
        }

        const answer = await response.json();
        if (!answer.sdp) {
            throw new Error("No SDP in answer");
        }

        await pc.setRemoteDescription(answer);
        console.log("✅ WebRTC streaming started");
        return pc; // success
    } catch (err) {
        console.error("❌ WebRTC setup failed:", err);
        pc.close();
        return null;
    }
})()
```

In [1]:
import asyncio
import threading
import cv2
import numpy as np
from aiohttp import web
from aiortc import RTCPeerConnection, RTCSessionDescription
from aiortc.mediastreams import VideoStreamTrack

# --- Shared frame storage ---
latest_frame = None
frame_lock = threading.Lock()
stop_display = threading.Event()


# --- WebRTC track receiver ---
class VideoFrameTrack(VideoStreamTrack):
    kind = "video"

    def __init__(self, track):
        super().__init__()
        self.track = track

    async def recv(self):
        global latest_frame
        frame = await self.track.recv()
        img = frame.to_ndarray(format="bgr24")  # OpenCV BGR
        with frame_lock:
            latest_frame = img
        return frame


# --- Signaling handler (aiohttp) ---
pcs = set()


async def offer_handler(request):
    offer = await request.json()
    pc = RTCPeerConnection()
    pcs.add(pc)

    @pc.on("track")
    def on_track(track):
        if track.kind == "video":
            pc.addTrack(VideoFrameTrack(track))

    await pc.setRemoteDescription(RTCSessionDescription(
        sdp=offer["sdp"], type=offer["type"]
    ))
    answer = await pc.createAnswer()
    await pc.setLocalDescription(answer)

    # 👇 Add CORS header here
    response = web.json_response({
        "sdp": pc.localDescription.sdp,
        "type": pc.localDescription.type
    })
    response.headers["Access-Control-Allow-Origin"] = "*"  # or "http://localhost:5000" for tighter security
    return response


async def offer_options(request):
    response = web.Response()
    response.headers["Access-Control-Allow-Origin"] = "*"
    response.headers["Access-Control-Allow-Methods"] = "POST, OPTIONS"
    response.headers["Access-Control-Allow-Headers"] = "Content-Type"
    return response


# --- OpenCV display thread ---
def img_modder(img):
    return img


def display_loop():
    cv2.namedWindow("WebRTC Stream", cv2.WINDOW_NORMAL)
    while not stop_display.is_set():
        with frame_lock:
            img = latest_frame.copy() if latest_frame is not None else None
        if img is not None:
            try:
                cv2.imshow("WebRTC Stream", img_modder(img))
            except Exception as e:
                print(e)
                cv2.imshow("WebRTC Stream", img)
        if cv2.waitKey(30) & 0xFF == ord('q'):  # ~33 FPS
            break
    cv2.destroyAllWindows()


# --- Start everything ---
async def start_webrtc_receiver():
    app = web.Application()
    app.router.add_post("/offer", offer_handler)
    app.router.add_options("/offer", offer_options)
    runner = web.AppRunner(app)
    await runner.setup()
    site = web.TCPSite(runner, "127.0.0.1", 8081)  # ← different port!
    await site.start()
    print("WebRTC receiver ready at http://127.0.0.1:8081/offer")


# Launch WebRTC server (background asyncio task)
asyncio.create_task(start_webrtc_receiver())

# Launch OpenCV display in a daemon thread
display_thread = threading.Thread(target=display_loop, daemon=True)
display_thread.start()

print("✅ WebRTC receiver running on :8081\n✅ OpenCV window will appear when stream starts")


✅ WebRTC receiver running on :8081
✅ OpenCV window will appear when stream starts
WebRTC receiver ready at http://127.0.0.1:8081/offer


![](files/v1-01.png)

In [16]:
from PIL import Image
import numpy as np


def crop_relative(img: Image.Image, left: float, top: float, right: float, bottom: float) -> Image.Image:
    """
    Crop a rectangle from a PIL image using normalized coordinates (0.0 to 1.0).

    Parameters:
        img: PIL Image object
        left, top, right, bottom: floats in [0, 1], where (0,0) is top-left and (1,1) is bottom-right

    Returns:
        Cropped PIL Image
    """
    # Validate inputs
    for coord in (left, top, right, bottom):
        if not (0.0 <= coord <= 1.0):
            raise ValueError("All coordinates must be between 0.0 and 1.0")
    if left >= right or top >= bottom:
        raise ValueError("Invalid crop region: left < right and top < bottom required")

    # Get image dimensions
    width, height = img.size

    # Convert normalized coords to absolute pixel values
    left_px = int(left * width)
    top_px = int(top * height)
    right_px = int(right * width)
    bottom_px = int(bottom * height)

    # Perform crop
    return img.crop((left_px, top_px, right_px, bottom_px))


def img_modder(img):
    """
    Keep only pixels that are truly blackish (all R,G,B low) as black.
    Everything else (including red, blue, green, etc.) becomes white.
    Input:  img = (H, W, 3) RGB numpy array (uint8)
    Output: (H, W) binary image (0 = black, 255 = white), uint8
    """
    # Ensure RGB uint8
    if img.ndim != 3 or img.shape[2] != 3:
        raise ValueError("Input must be RGB image with shape (H, W, 3)")
    if img.dtype != np.uint8:
        img = img.astype(np.uint8)

    # Define max value for a channel to be considered "dark"
    CHANNEL_THRESHOLD = 40  # tune this (e.g., 30–50)

    # A pixel is blackish only if R < T AND G < T AND B < T
    blackish_mask = (
            (img[:, :, 0] < CHANNEL_THRESHOLD) &
            (img[:, :, 1] < CHANNEL_THRESHOLD) &
            (img[:, :, 2] < CHANNEL_THRESHOLD)
    )

    # Create output: 0 where blackish, 255 elsewhere
    binary = np.where(blackish_mask, 0, 255).astype(np.uint8)

    return binary

![](files/v1-02.png)

In [3]:
(700, 897, 3)

KeyboardInterrupt: 